<a href="https://colab.research.google.com/github/Vietanh-Vu/Machine-learning-algorithm/blob/main/Detect_Comment_Social_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install --upgrade git+https://github.com/jpmml/sklearn2pmml.git

  Cloning https://github.com/jpmml/sklearn2pmml.git to /tmp/pip-req-build-o4tfng1k
  Running command git clone --filter=blob:none --quiet https://github.com/jpmml/sklearn2pmml.git /tmp/pip-req-build-o4tfng1k
  Resolved https://github.com/jpmml/sklearn2pmml.git to commit 3ab1489f7cd421cd57652c4f83743f4ed9ba211e
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.6 MB/s eta 0:00:00
  Created wheel for sklearn2pmml: filename=sklearn2pmml-0.110.0-py3-none-any.whl size=7181295 sha256=cbeaa09292d4b54c57b562c940184d1537950c80ffb7a7aa57d87f710be90be2
  Stored in directory: /tmp/pip-ephem-wheel-cache-rfg4pjld/wheels/85/30/d8/67de0c17ff5064a0f831858103b20643767323941824cb96e3
Successfully built sklearn2pmml


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import metrics
import joblib
from sklearn2pmml import sklearn2pmml
from sklearn2pmml.pipeline import PMMLPipeline
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
train = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/Comment/train-comment.csv')
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26799 entries, 0 to 26798
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   free_text  26797 non-null  object
 1   label_id   26799 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 418.9+ KB


In [5]:
validate = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/Comment/validate-comment.csv')
validate.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2672 entries, 0 to 2671
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   free_text  2672 non-null   object
 1   label_id   2672 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 41.9+ KB


In [6]:
test = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/Comment/test-comment.csv')
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6680 entries, 0 to 6679
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   free_text  6680 non-null   object
 1   label_id   6680 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 104.5+ KB


In [7]:
train['free_text'].fillna('', inplace=True)
validate['free_text'].fillna('', inplace=True)
test['free_text'].fillna('', inplace=True)

In [8]:
X_train = train['free_text']
y_train = train['label_id']
X_test = test['free_text']
y_test = test['label_id']

In [9]:
pipeline = PMMLPipeline([
    ("vectorizer", CountVectorizer()),
    ('classifier', LogisticRegression())
])

In [10]:
pipeline.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


PMMLPipeline(steps=[('vectorizer', CountVectorizer()),
       ('classifier', LogisticRegression())])

In [11]:
# Make predictions
y_pred = pipeline.predict(X_test)
print("Accuracy:", metrics.accuracy_score(y_test, y_pred))

Accuracy: 0.8907185628742516


In [12]:
# Save the model to PMML
sklearn2pmml(pipeline, "/content/drive/MyDrive/ColabNotebooks/Comment/comment-model.pmml", with_repr=True)

In [13]:
!pip install pypmml
from pypmml import Model

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.2/14.2 MB 81.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pypmml: filename=pypmml-0.9.17-py3-none-any.whl size=14215034 sha256=9222038aca1b189905a9729c4e4b2db487136ade35f108da197eed6b0f877e74
  Stored in directory: /root/.cache/pip/wheels/8c/74/f1/946a04acaa6de2e9df0f02739511aba5a7aac52383c52ac900
Successfully built pypmml


In [14]:

model = Model.load("/content/drive/MyDrive/ColabNotebooks/Comment/comment-model.pmml")
def predict_comment(comment):
    # Prepare the input
    input_df = pd.DataFrame({'free_text': [comment]})

    # Make prediction
    prediction = model.predict(input_df)
    return prediction
    print(f"% Not spam: {prediction['probability(0)'][0]}")
    print(f"% Spam: {prediction['probability(1)'][0]}")


# Test the model with a sample comment
sample_comment = "Mày chỉ đáng con chó, không hơn không kém"
prediction = predict_comment(sample_comment)
prediction

,probability(0),probability(1)
0,0.132993,0.867007
